In [1]:
!pip -q install -U "openai>=1.54.0" datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.0/812.0 kB 33.1 MB/s eta 0:00:00


In [2]:
import os
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"  # Stable identifier (Flash-Lite)
REASONING_EFFORT = "low"            # Minimal reasoning per your request

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)

print("Client ready with OpenAI-compatible Gemini endpoint.")


Client ready with OpenAI-compatible Gemini endpoint.


In [3]:
from datasets import load_dataset

ds = load_dataset("mohammad-shirkhani/social_movielens_custom", split="train")
print(ds)
print("Rows:", len(ds))
print("Columns:", ds.column_names)

ex0 = ds[0]
print("\nFirst row keys:", list(ex0.keys()))

for k, v in ex0.items():
    if isinstance(v, list) and len(v) > 3:
        print(f"- {k}: list(len={len(v)}), first 2:\n  {v[:2]}")
    else:
        print(f"- {k}: {v}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/691 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/29.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset({
    features: ['user', 'item', 'answer', 'paths'],
    num_rows: 20000
})
Rows: 20000
Columns: ['user', 'item', 'answer', 'paths']

First row keys: ['user', 'item', 'answer', 'paths']
- user: {'UserID': 911, 'Age': 37, 'Gender': 'Female', 'Occupation': 'writer'}
- item: {'MovieID': 193, 'Title': 'Right Stuff, The (1983)', 'ReleaseDate': '01-Jan-1983', 'Genres': 'Drama'}
- answer: 4.0
- paths: list(len=20), first 2:
  ['user_question -> user_item_2 (rating=2) -> Item{MovieID 98, Title "Silence of the Lambs, The (1991)", Release Date 01-Jan-1991, Genres Drama, Thriller} -> item_user_4 (rating=4) -> User{UserID 498, Age 26, Gender Male, Occupation writer} -> usersim -> User{UserID 58, Age 27, Gender Male, Occupation programmer} -> user_item_3 (rating=3) -> item_question', 'user_question -> usersim -> User{UserID 716, Age 36, Gender Female, Occupation administrator} -> user_item_5 (rating=5) -> Item{MovieID 517, Title "Manhattan (1979)", Release Date 01-Jan-1979, Genres Comedy, D

In [4]:
from typing import Dict, Any

def detect_fields(example: Dict[str, Any]):
    user_keys = ["user", "User", "user_dict", "UserDict"]
    item_keys = ["item", "Item", "item_dict", "ItemDict"]
    rating_keys = ["answer", "rating", "label", "score"]
    paths_keys = ["paths", "meta_paths", "metapaths", "path_list", "path", "metaPaths"]

    def find_key(candidates, predicate=None):
        for k in candidates:
            if k in example and (predicate(example[k]) if predicate else True):
                return k
        for k, v in example.items():
            if predicate and predicate(v):
                return k
        return None

    user_key = find_key(user_keys, predicate=lambda v: isinstance(v, dict))
    item_key = find_key(item_keys, predicate=lambda v: isinstance(v, dict))
    rating_key = find_key(rating_keys, predicate=lambda v: isinstance(v, (float, int)))
    paths_key = find_key(paths_keys, predicate=lambda v: isinstance(v, list))

    if user_key is None:
        user_key = next((k for k, v in example.items() if isinstance(v, dict)), None)
    if item_key is None:
        cand = [k for k, v in example.items() if isinstance(v, dict) and k != user_key]
        item_key = cand[0] if cand else None
    if rating_key is None:
        cand = [k for k, v in example.items() if isinstance(v, (float, int))]
        rating_key = cand[0] if cand else None
    if paths_key is None:
        cand = [k for k, v in example.items() if isinstance(v, list)]
        paths_key = cand[0] if cand else None

    return user_key, item_key, rating_key, paths_key

user_key, item_key, rating_key, paths_key = detect_fields(ex0)
print("Detected keys ->",
      "user:", user_key,
      "| item:", item_key,
      "| rating:", rating_key,
      "| paths:", paths_key)

user0 = ex0[user_key]
item0 = ex0[item_key]
rating0 = ex0[rating_key]
paths0 = ex0[paths_key]
print("\nUser example:", user0)
print("Item example:", item0)
print("Rating example:", rating0)
print("Meta-paths:", len(paths0))

Detected keys -> user: user | item: item | rating: answer | paths: paths

User example: {'UserID': 911, 'Age': 37, 'Gender': 'Female', 'Occupation': 'writer'}
Item example: {'MovieID': 193, 'Title': 'Right Stuff, The (1983)', 'ReleaseDate': '01-Jan-1983', 'Genres': 'Drama'}
Rating example: 4.0
Meta-paths: 20


In [5]:
from typing import List

def format_kv_block(title: str, d: Dict[str, Any]) -> str:
    lines = [f"{title}:"]
    for k, v in d.items():
        lines.append(f"- {k}: {v}")
    return "\n".join(lines)

def format_meta_paths(paths: List[str]) -> str:
    lines = ["Meta-path evidence (each path from this user to the target item):"]
    for p in paths:
        p = p.strip()
        if not p.startswith("- "):
            p = "- " + p
        lines.append(p)
    return "\n".join(lines)

def build_prompt_evidence_then_answer(
    user: Dict[str, Any],
    item: Dict[str, Any],
    rating: float,
    meta_paths: List[str],
) -> str:
    user_block = format_kv_block("User", user)
    item_block = format_kv_block("Item", item)
    observed_rating_str = str(rating)
    meta_block = format_meta_paths(meta_paths)

    instruction = (
        "Task: Extract evidence from the provided data and then conclude the numerical rating.\n\n"
        "You are given a heterogeneous bipartite graph setting (users and items). Edges include:\n"
        "- user→item rating interactions (e.g., user_item_k with an explicit rating),\n"
        "- usersim (user-user similarity), and\n"
        "- itemsim (item-item similarity).\n\n"
        "What to do:\n"
        "1) Analyze the user's likely preferences and the item's traits by leveraging ONLY:\n"
        "   - the user attributes,\n"
        "   - the item attributes, and\n"
        "   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).\n"
        "   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).\n"
        "2) Then produce two XML blocks ONLY (no extra text):\n"
        "   a) <reason>...</reason> — Provide a clear, evidence-first explanation that states\n"
        "      what information you obtain from the user/item/meta-paths and how those signals combine.\n"
        "      Avoid wording like “because the rating is X”. Instead, present evidence → inference, and end with a neutral\n"
        "      sentence such as: “Therefore, the rating equals {R}.”\n"
        "   b) <answer>{R}</answer> — Put the observed rating number {R} exactly as given below.\n\n"
        "STRICT FORMAT RULES:\n"
        "- Output ONLY these two blocks in this order, nothing else:\n"
        "<reason>\n"
        "...your evidence-first explanation here...\n"
        "</reason>\n"
        "<answer>{R}</answer>\n"
        "- Do not invent attributes, paths, or ratings not present in the input. If something is unknown, treat it as unknown.\n"
    )

    observed_block = f"Observed rating (user → item): {observed_rating_str}"
    prompt = (
        instruction
        + "\n=== INPUT ===\n"
        + user_block + "\n\n"
        + item_block + "\n\n"
        + observed_block + "\n\n"
        + meta_block + "\n"
        + "=== END INPUT ===\n"
    )
    return prompt.replace("{R}", observed_rating_str)

prompt0 = build_prompt_evidence_then_answer(user0, item0, rating0, paths0)
print("Prompt:\n")
print(prompt0[:])


Prompt:

Task: Extract evidence from the provided data and then conclude the numerical rating.

You are given a heterogeneous bipartite graph setting (users and items). Edges include:
- user→item rating interactions (e.g., user_item_k with an explicit rating),
- usersim (user-user similarity), and
- itemsim (item-item similarity).

What to do:
1) Analyze the user's likely preferences and the item's traits by leveraging ONLY:
   - the user attributes,
   - the item attributes, and
   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).
   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).
2) Then produce two XML blocks ONLY (no extra text):
   a) <reason>...</reason> — Provide a clear, evidence-first explanation that states
      what information you obtain from the user/item/meta-paths and how those signals combine.
      Avoid wording like “because the rating is X”. In

In [6]:
import re

def call_gemini(prompt: str, max_tokens: int = 2048):
    resp = client.chat.completions.create(
        model=MODEL_ID,
        reasoning_effort=REASONING_EFFORT,  
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        n=1,
    )
    return resp.choices[0].message.content

def parse_reason_answer(text: str):
    reason = None
    answer = None
    m_reason = re.search(r"<reason>(.*?)</reason>", text, flags=re.DOTALL | re.IGNORECASE)
    if m_reason:
        reason = m_reason.group(1).strip()
    m_answer = re.search(r"<answer>\s*([0-9]+(?:\.[0-9]+)?)\s*</answer>", text, flags=re.DOTALL | re.IGNORECASE)
    if m_answer:
        answer = m_answer.group(1).strip()
    return reason, answer

In [7]:
import time
import math
from collections import deque

RPM_LIMIT = 15          
TPM_LIMIT = 250_000      
OUT_TOKENS_BUDGET = 2048  

class RateLimiter:
    def __init__(self, rpm: int, tpm: int):
        self.rpm = rpm
        self.tpm = tpm
        self.req_times = deque()
        self.token_times = deque()  # (timestamp, tokens)

    @staticmethod
    def estimate_tokens(text: str) -> int:
        return max(1, math.ceil(len(text) / 4))

    def wait(self, prompt_text: str, out_tokens:int = OUT_TOKENS_BUDGET):
        now = time.time()
        while self.req_times and now - self.req_times[0] > 60:
            self.req_times.popleft()
        while self.token_times and now - self.token_times[0][0] > 60:
            self.token_times.popleft()

        in_tokens = self.estimate_tokens(prompt_text)
        while len(self.req_times) >= self.rpm:
            sleep_s = 60 - (now - self.req_times[0])
            time.sleep(max(0.01, sleep_s))
            now = time.time()
            while self.req_times and now - self.req_times[0] > 60:
                self.req_times.popleft()

        used_tpm = sum(t for (_, t) in self.token_times)
        while used_tpm + in_tokens + out_tokens > self.tpm:
            sleep_s = 60 - (now - self.token_times[0][0])
            time.sleep(max(0.01, sleep_s))
            now = time.time()
            while self.token_times and now - self.token_times[0][0] > 60:
                self.token_times.popleft()
            used_tpm = sum(t for (_, t) in self.token_times)

        self.req_times.append(time.time())
        self.token_times.append((time.time(), in_tokens + out_tokens))

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

def run_one_example(example, max_tokens:int=2048):
    user = example[user_key]
    item = example[item_key]
    rating = example[rating_key]
    meta = example[paths_key]

    prompt = build_prompt_evidence_then_answer(user, item, rating, meta)

    limiter.wait(prompt, out_tokens=max_tokens)

    backoff = 1.0
    for attempt in range(5):
        try:
            text = call_gemini(prompt, max_tokens=max_tokens)
            reason, answer = parse_reason_answer(text)
            return text, reason, answer
        except Exception as e:
            if attempt == 4:
                raise
            time.sleep(backoff)
            backoff *= 2.0


In [ ]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 16700
END_INDEX   = 17600   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_16700_17599.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 16700..17599, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 16700..17599).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 16700..17599, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/9…


Done (rows 16700..17599).
OK: 884 | Missing reason: 12 | Missing answer: 13 | Failures: 3
Saved to: /content/gemini_flash_lite_reason_answer_16700_17599.jsonl

Sample outputs:
{'index': 16700, 'user': {'UserID': 655, 'Age': 50, 'Gender': 'Female', 'Occupation': 'healthcare'}, 'item': {'MovieID': 516, 'Title': 'Local Hero (1983)', 'ReleaseDate': '01-Jan-1983', 'Genres': 'Comedy'}, 'rating': 2.0, 'reason': 'The user, UserID 655, is 50 years old, Female, and works in healthcare. The target item, MovieID 516, is "Local Hero (1983)", a Comedy.\n\nAnalysis of meta-paths reveals several weak signals:\n1.  A path shows User 655 rating an item (MovieID 459, Genres: Drama) with 2.0. This suggests a potential for lower ratings on certain genres.\n2.  Another path shows User 655 rating an item (MovieID 764, Genres: Comedy, Romance) with 1.0. This provides a stronger indication of a negative preference, especially for comedy/romance.\n3.  A path shows User 655 rating an item (MovieID 653, Genres: 

In [ ]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 17600
END_INDEX   = 18500   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_17600_18499.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 17600..18499, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 17600..18499).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 17600..18499, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/9…


Done (rows 17600..18499).
OK: 876 | Missing reason: 24 | Missing answer: 24 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_17600_18499.jsonl

Sample outputs:
{'index': 17600, 'user': {'UserID': 903, 'Age': 28, 'Gender': 'Male', 'Occupation': 'educator'}, 'item': {'MovieID': 931, 'Title': 'Island of Dr. Moreau, The (1996)', 'ReleaseDate': '23-Aug-1996', 'Genres': 'Sci-Fi, Thriller'}, 'rating': 2.0, 'reason': 'The user (UserID 903) is a 28-year-old male educator. The target item is "Island of Dr. Moreau, The (1996)", categorized as Sci-Fi and Thriller.\n\nSeveral meta-paths suggest a preference for genres or items rated lower by similar users or connected users.\nSpecifically, user 214 (a librarian, rated items a 2 and 4) is connected via `usersim` and then rated an item (Drama, Romance) a 2. User 214 is also connected via `usersim` to another item (Action, Sci-Fi, Thriller) which received a rating of 4 from that user, but subsequently a rating of 2 from a user (UserID

In [8]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 18500
END_INDEX   = 19400   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_18500_19399.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 18500..19399, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 18500..19399).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 18500..19399, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/9…


Done (rows 18500..19399).
OK: 885 | Missing reason: 15 | Missing answer: 15 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_18500_19399.jsonl

Sample outputs:
{'index': 18500, 'user': {'UserID': 472, 'Age': 24, 'Gender': 'Male', 'Occupation': 'student'}, 'item': {'MovieID': 977, 'Title': 'Substitute, The (1996)', 'ReleaseDate': '19-Apr-1996', 'Genres': 'Action'}, 'rating': 3.0, 'reason': 'The user (UserID 472) is a 24-year-old male student, and the item (MovieID 977) is an Action genre movie. Analysis of the meta-paths reveals several patterns. Multiple paths involve other users with the \'student\' occupation (UserID 94, UserID 276, UserID 393, UserID 159, UserID 223, UserID 851, UserID 727) interacting with items across various genres, including Action. These interactions show a mix of ratings, with a notable presence of 3.0 ratings (e.g., UserID 393 -> MovieID 391 (Action/Comedy) rating=4, UserID 378 -> MovieID 80 (Action/Comedy/War) rating=3, UserID 279 -> MovieID

In [9]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 19400
END_INDEX   = 20000   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_19400_19999.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 19400..19999, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 19400..19999).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 19400..19999, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/6…


Done (rows 19400..19999).
OK: 588 | Missing reason: 11 | Missing answer: 12 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_19400_19999.jsonl

Sample outputs:
{'index': 19400, 'user': {'UserID': 840, 'Age': 39, 'Gender': 'Male', 'Occupation': 'artist'}, 'item': {'MovieID': 653, 'Title': 'Touch of Evil (1958)', 'ReleaseDate': '01-Jan-1958', 'Genres': 'Crime, Film-Noir, Thriller'}, 'rating': 5.0, 'reason': 'The user, identified as an artist aged 39, is connected to the target item, "Touch of Evil (1958)" (Genres: Crime, Film-Noir, Thriller), through several meta-paths. Notably, multiple paths indicate strong positive interactions with items that share thematic or genre similarities with "Touch of Evil". For instance, user 840 exhibits a rating of 5 for "Silence of the Lambs, The (1991)" (Genres: Drama, Thriller) and "Taxi Driver (1976)" (Genres: Drama, Thriller). Additionally, "Sunset Blvd. (1950)" (Genres: Film-Noir), which is similar via `itemsim` to "Annie Hall (1977